# Assignment 7

## Submit as an HTML file

### Print your name below

In [75]:
print("Zoe Liu")

Zoe Liu


### Import the "pandas" "numpy" and "statsmodels.formula.api" libraries

In [76]:
# Write your answer here:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

#### In the code chunk below read the CSV file named `results.csv` in the `data` folder and print the first 5 rows of the dataset. Browse the dataset.

In [77]:
results = pd.read_csv("data/results.csv")
results.iloc[0:5 , : ]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


### (a)  Check Column Types and Data Cleaning

- Use the function .dtypes to get the column types
- Identify which columns have data types that might need conversion
- The 'milliseconds' column contains string values that should be numeric. Create a new column called 'race_time_ms' that:
    - Converts the column to a numeric data type
    - Replaces any non-numeric values with NaN

In [78]:
# Write your answer here
print(results.dtypes)
# number, position, positionText, milliseconds, fastestLap, rank, and fastestLapSpeed can be converted into numeric types (int or float) instead of being string values.
results["milliseconds"] = results["milliseconds"].replace('\\N', np.nan)
results["race_time_ms"] = pd.to_numeric(results["milliseconds"])
results

resultId             int64
raceId               int64
driverId             int64
constructorId        int64
number              object
grid                 int64
position            object
positionText        object
positionOrder        int64
points             float64
laps                 int64
time                object
milliseconds        object
fastestLap          object
rank                object
fastestLapTime      object
fastestLapSpeed     object
statusId             int64
dtype: object


,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId,race_time_ms
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1,5690616.0
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1,5696094.0
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1,5698779.0
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1,5707797.0
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1,5708630.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25835,25841,1096,854,210,47,12,16,16,16,0.0,57,\N,NaN,39,12,1:29.833,211.632,11,NaN
25836,25842,1096,825,210,20,16,17,17,17,0.0,57,\N,NaN,40,20,1:31.158,208.556,11,NaN
25837,25843,1096,1,131,44,5,18,18,18,0.0,55,\N,NaN,42,11,1:29.788,211.738,9,NaN
25838,25844,1096,849,3,6,20,19,19,19,0.0,55,\N,NaN,45,14,1:30.309,210.517,130,NaN


### (b) Create Categorical Variables

- Create a new column called 'finish_category' that categorizes the race finish positions as follows:
    - Positions 1-3: 'Podium'
    - Positions 4-10: 'Points'
    - Positions 11-20: 'Midfield'
    - Positions >20: 'Backmarker'

Hint: Use the pd.cut() function

In [79]:
# Write your answer here
results["position"] = results["position"].replace('\\N', np.nan)
results["position"] = pd.to_numeric(results["position"])
bins_num = (1, 4, 11, 21, float("inf"))
labels_pos = ["Podium", "Points", "Midfield", "Backmarker"]
results["finish_category"] = pd.cut(results["position"],
                                    bins = bins_num,
                                    right = False,
                                    labels = labels_pos)
results[["position", "finish_category"]]

,position,finish_category
0,1.0,Podium
1,2.0,Podium
2,3.0,Podium
3,4.0,Points
4,5.0,Points
...,...,...
25835,16.0,Midfield
25836,17.0,Midfield
25837,18.0,Midfield
25838,19.0,Midfield


### (c) Calculate Race Duration
- For rows where 'milliseconds' is available, create a new column 'race_duration_minutes' that converts milliseconds to minutes by dividing by (1000*60).
- Display the average race duration by 'constructorId' for the top 5 constructors with the shortest average race times

In [80]:
# Write your answer here
results["race_duration_minutes"] = results["race_time_ms"]/(1000 * 60)
top5 = (results.groupby("constructorId")
        .agg(mean_duration = ("race_duration_minutes","mean"))
        .sort_values(by = "mean_duration", ascending = True)
        .iloc[0:5 , : ])
top5

,mean_duration
constructorId,
35,76.710777
29,77.604125
41,87.046767
16,89.428828
53,89.658852


### (d) Driver Performance Analysis

- Calculate the following statistics for each driver, grouped by 'driverId':
    - Average finishing position
    - Total points
    - Number of races completed
    - Best finishing position

- Sort the results by total points in descending order
- Display the top 10 drivers based on total points

In [81]:
# Write your answer here
drivers_agg = (results.groupby("driverId")
               .agg(avg_pos = ("position", "mean"),
                    total_pt = ("points", "sum"),
                    num_race = ("driverId", len),
                    best_pos = ("position", "min"))
               .sort_values(by = "total_pt", ascending = False)
               .iloc[0:10 , : ])
drivers_agg

,avg_pos,total_pt,num_race,best_pos
driverId,,,,
1,3.383803,4396.5,310,1.0
20,5.282443,3098.0,300,1.0
4,6.257732,2061.0,358,1.0
830,4.022388,1983.5,163,1.0
8,6.238596,1873.0,352,1.0
822,6.296089,1778.0,201,1.0
3,6.355932,1594.5,206,1.0
30,3.701245,1566.0,308,1.0
817,8.261538,1307.0,232,1.0


### (e) Linear Regression
Create a linear regression model that predicts 'points' based on 'grid' (starting position) and 'laps' completed <br>
Use the following steps:

- Clean the data to remove any non-numeric values and missing values
- Create the regression formula using smf.ols 
- Display the summary of the regression model using model.summary()

What is the predicted points for a driver starting in position 3 and completing 55 laps?

Hint: Use ```.dropna()''' to remove missing values from the points, grid, and laps variables.

In [82]:
# Write your answer here
reg_data = results[["points", "grid", "laps"]].dropna()
model = smf.ols(formula = "points ~ grid + laps", data = reg_data)
results_fit = model.fit()
print(results_fit.summary())
b_list = results_fit.params
print(b_list)
predicted_points = b_list["Intercept"] + b_list["grid"] * 3 + b_list["laps"] * 55
print("Predicted Points:", predicted_points)

                            OLS Regression Results                            
Dep. Variable:                 points   R-squared:                       0.215
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     3530.
Date:                Wed, 19 Mar 2025   Prob (F-statistic):               0.00
Time:                        19:20:29   Log-Likelihood:                -70440.
No. Observations:               25840   AIC:                         1.409e+05
Df Residuals:                   25837   BIC:                         1.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.5841      0.054     48.267      0.0